In [1]:
!pip install cassandra-driver

In [4]:
from cassandra.cluster import Cluster
from ssl import SSLContext, PROTOCOL_TLSv1_2 , CERT_REQUIRED
from cassandra.auth import PlainTextAuthProvider

In [5]:
cluster_cassandra = Cluster(['127.0.0.1'], port=9042)

In [6]:
session_cassandra = cluster_cassandra.connect()

In [7]:
command = """
CREATE KEYSPACE IF NOT EXISTS sourceu18

WITH replication = {'class':'SimpleStrategy', 'replication_factor' : 2};
"""

In [8]:
session_cassandra.execute(command)

In [9]:
command = """
CREATE TABLE IF NOT EXISTS sourceu18.employees (
   no int PRIMARY KEY,
   name text)
"""

In [10]:
session_cassandra.execute(command)

In [11]:
command = """
USE sourceu18
"""

In [12]:
session_cassandra.execute(command)

In [13]:
command = """
insert into sourceu18.employees (no, name) values(1, 'Shrini')
"""

In [14]:
session_cassandra.execute(command)

In [15]:
!pip install pandas
import pandas as pd

In [18]:
dfemp = pd.read_csv("employees.csv")

In [19]:
dfemp

,no,name
0,1,Atin
1,2,Chetan
2,3,Badri
3,4,Satish
4,5,Suresh
5,6,Sandeep
6,7,Manish
7,8,Parul
8,9,Pooja
9,10,Jaynesh


In [46]:
for emp in dfemp.iterrows():
    emp_no = emp[1][0]
    emp_name = emp[1][1]
    command = f"""
    insert into sourceu18.employees (no, name) values({emp_no}, '{emp_name}')
    """
    session_cassandra.execute(command)

/tmp/ipykernel_2321/2654020525.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  emp_no = emp[1][0]
/tmp/ipykernel_2321/2654020525.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  emp_name = emp[1][1]


In [47]:
r = session_cassandra.execute('select * from sourceu18.employees')
print(r.current_rows)

[Row(no=5, name='Suresh'), Row(no=10, name='Jaynesh'), Row(no=1, name='Atin'), Row(no=8, name='Parul'), Row(no=2, name='Chetan'), Row(no=4, name='Satish'), Row(no=7, name='Manish'), Row(no=6, name='Sandeep'), Row(no=9, name='Pooja'), Row(no=3, name='Badri')]


In [50]:
ssl_context = SSLContext(PROTOCOL_TLSv1_2 )
ssl_context.load_verify_locations('sf-class2-root.crt')

/tmp/ipykernel_2321/972152945.py:1: DeprecationWarning: ssl.PROTOCOL_TLSv1_2 is deprecated
  ssl_context = SSLContext(PROTOCOL_TLSv1_2 )


In [56]:
ssl_context.verify_mode = CERT_REQUIRED
auth_provider = PlainTextAuthProvider(username='u1-at-044323040668', password='2/XrmsY7nIBQJCl0KBgZjhWY9P/SYM3DRPZ4w8dopRI=')

In [57]:
cluster_aws = Cluster(['cassandra.us-east-1.amazonaws.com'], ssl_context=ssl_context, auth_provider=auth_provider, port=9142)

In [58]:
session_aws = cluster_aws.connect()

In [59]:
command = """
select * from sourceu18.employees
"""

In [60]:
rows_source_emp = session_cassandra.execute(command)

In [62]:
from cassandra import ConsistencyLevel
from cassandra.query import SimpleStatement

for row in rows_source_emp:
    query = SimpleStatement(
        "INSERT INTO  target.employees (no, name) VALUES (%s, %s)",
        consistency_level=ConsistencyLevel.LOCAL_QUORUM)
    session_aws.execute(query, row)

In [63]:
rows = session_aws.execute("select * from target.employees")

In [64]:
for row in rows:
    print(row)

Row(no=2, name='Chetan')
Row(no=8, name='Parul')
Row(no=9, name='Pooja')
Row(no=7, name='Manish')
Row(no=6, name='Sandeep')
Row(no=5, name='Suresh')
Row(no=4, name='Satish')
Row(no=1, name='Atin')
Row(no=3, name='Badri')
Row(no=10, name='Jaynesh')
